<a href="https://colab.research.google.com/github/Nikoschenk/language_model_finetuning/blob/master/gpt2_fine_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetuning Transformers on [CORD-19](https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge)
This notebook demonstrates how to fine-tune GPT-2 on the [COVID-19 Open Research Dataset (CORD-19)](https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge) using the [Hugging Face Transformer](https://github.com/huggingface/transformers) library, and includes examples for text generation comparing both the base model and the fine-tuned model.

Credits: Modification of [original notebook](https://colab.research.google.com/github/interactive-fiction-class/interactive-fiction-class.github.io/blob/master/homeworks/language-model/hw4_transformer.ipynb) provided by [Daphne Ippolito](https://www.seas.upenn.edu/~daphnei/)


## Installation

### Install the HuggingFace Transfomers library.

In [0]:
# Install transformers.
!git clone https://github.com/huggingface/transformers

import os
os.chdir('/content/transformers')

# Use language modeling version as of April 21st.
!git checkout b1ff0b2ae7d368b7db3a8a8472a29cc195d278d8

!pip install .
!pip install -r ./examples/requirements.txt

os.chdir('/content/transformers/examples')

!pip install dict_to_obj

In [0]:
import torch
import run_language_modeling
import run_generation
from dict_to_obj import DictToObj
import collections
import random
import numpy as np

from transformers import AutoConfig
from transformers import AutoTokenizer
from transformers import AutoModelWithLMHead

# Make sure that this version of transformers has the correct evaluate functionality.
from run_language_modeling import evaluate

### Mount your Google Drive
Checkpoints will be saved Google Drive folders.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

### Download CORD-19 data.
We provide dummy data here

In [0]:
# Download the train and test set.
!wget -nc -O /content/presidential_speeches_test.txt https://raw.githubusercontent.com/interactive-fiction-class/interactive-fiction-class.github.io/master/homeworks/language-model/presidential_speeches_test.txt
!wget -nc -O /content/presidential_speeches_valid.txt https://raw.githubusercontent.com/interactive-fiction-class/interactive-fiction-class.github.io/master/homeworks/language-model/presidential_speeches_valid.txt
!wget -nc -O /content/presidential_speeches_train.txt https://raw.githubusercontent.com/interactive-fiction-class/interactive-fiction-class.github.io/master/homeworks/language-model/presidential_speeches_train.txt

## Finetune and evaluate

### Launch fine-tuninng


In [0]:
# GPT-2 fine-tuning.
!python run_language_modeling.py \
    --output_dir='/content/drive/My Drive/finetuned_models/presidential_speeches' \
    --model_type=gpt2 \
    --model_name_or_path=gpt2-medium \
    --save_total_limit=5 \
    --num_train_epochs=1.0 \
    --do_train \
    --evaluate_during_training \
    --logging_steps=500 \
    --save_steps=500 \
    --train_data_file=/content/presidential_speeches_train.txt \
    --do_eval \
    --eval_data_file=/content/presidential_speeches_valid.txt \
    --per_gpu_train_batch_size=2 \
    --per_gpu_eval_batch_size=2 \
    --block_size=128 \
    --gradient_accumulation_steps=5

### Compute perplexity of a dataset.


#### Look at what checkpoints are available


In [0]:
!ls '/content/drive/My Drive/finetuned_models/presidential_speeches'

#### Helper functions

In [0]:
def load_model(args):
  """Creates a model and loads in weights for it."""
  config = AutoConfig.from_pretrained(args.model_name_or_path, cache_dir=None)

  model = AutoModelWithLMHead.from_pretrained(
      args.model_name_or_path,
      from_tf=bool(".ckpt" in args.model_name_or_path),
      config=config,
      cache_dir=None
  )
  
  model.to(args.device)
  return model

def set_seed(seed):
  """Set the random seed."""
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if args.n_gpu > 0:
    torch.cuda.manual_seed_all(args.seed)

def do_perplexity_eval(args, model, data_file_path):
  """Computes the perplexity of the text in data_file_path according to the provided model."""
  set_seed(args.seed)

  args.eval_data_file=data_file_path

  tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, cache_dir=None)

  args.block_size = min(args.block_size, tokenizer.max_len)

  result = run_language_modeling.evaluate(args, model, tokenizer, prefix="")
  return result

#### Compute it.

In [0]:
# GPT-2 EVALUATION:

# 1. Fine-tuned model.
CHECKPOINT_PATH = '/content/drive/My Drive/finetuned_models/presidential_speeches/checkpoint-3000'

# 2. Non-fine-tuned model.
#CHECKPOINT_PATH = "gpt2-medium" 

# Set this to the list of text files you want to evaluate the perplexity of.
DATA_PATHS = ["/content/presidential_speeches_valid.txt",
              "/content/presidential_speeches_test.txt"]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("Running on device: ", device)

args = collections.defaultdict(
  model_name_or_path=CHECKPOINT_PATH,
  output_dir=CHECKPOINT_PATH,
  block_size = 128,
  local_rank=-1,
  eval_batch_size=2,
  per_gpu_eval_batch_size=2,
  n_gpu=n_gpu,
  mlm=False,
  device=device,
  line_by_line=False,
  overwrite_cache=None,
  model_type='gpt2',
  seed=42,
)
args = DictToObj(args)

model = load_model(args)

for data_path in DATA_PATHS:
  eval_results = do_perplexity_eval(args, model, data_path)
  perplexity = eval_results['perplexity']
  print('{} is the perplexity of {} according to {}'.format(
      perplexity, data_path, CHECKPOINT_PATH))

### Generate samples


In [0]:
def generate_samples(args, model, prompt_text):
  """Generating sampling for the provided prompt using the provided model."""
  set_seed(args.seed)

  tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, cache_dir=None)

  requires_preprocessing = args.model_type in run_generation.PREPROCESSING_FUNCTIONS.keys()
  encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors="pt")
  encoded_prompt = encoded_prompt.to(args.device)

  output_sequences = model.generate(
      input_ids=encoded_prompt,
      max_length=args.length + len(encoded_prompt[0]),
      temperature=args.temperature,
      top_k=args.k,
      top_p=args.p,
      repetition_penalty=args.repetition_penalty,
      do_sample=True,
      num_return_sequences=args.num_return_sequences,
  )

  # Remove the batch dimension when returning multiple sequences
  if len(output_sequences.shape) > 2:
    output_sequences.squeeze_()

  generated_sequences = []

  for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()

    # Decode text
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

    # Remove all text after the stop token
    text = text[: text.find(args.stop_token) if args.stop_token else None]

    # Remove the excess text that was used for pre-processing
    text = text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]

    # Add the prompt at the beginning of the sequence.
    total_sequence = prompt_text + text

    generated_sequences.append(total_sequence)

  return generated_sequences

In [0]:
# GPT-2 text generation:

# Set this to the checkpoint you want to use for generation, or to "gpt2-medium"
# to generate with the pre-trained model without finetuning.
CHECKPOINT_PATH = '/content/drive/My Drive/finetuned_models/presidential_speeches/checkpoint-1500'

# You should try out other prompts as well as no prompt at all.
PROMPT = 'Add a prompt here.'


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("Running on device: ", device)

args = collections.defaultdict(
  model_name_or_path=CHECKPOINT_PATH,
  output_dir=CHECKPOINT_PATH,
  n_gpu=n_gpu,
  mlm=False,
  device=device,
  model_type='gpt2',
  seed=42,
  stop_token=None, # Set this if your dataset has a special word that indicates the end of a text.
  temperature=1.0,  # temperature sampling. Set this to temperature=1.0 to not use temperature.
  k=50,  # k for top-k sampling. Set this to k=0 to not use top-k.
  p=1.0,  # p for nucleus sampling. Set this to p=1.0 to not use nucleus sampling.
  repetition_penalty=None,
  length=100,  # Number of tokens to generate.
  num_return_sequences=3,  # Number of independently computed samples to generate.
)
args = DictToObj(args)

model = load_model(args)
sequences = generate_samples(args, model, PROMPT)
for idx, sequence in enumerate(sequences):
  print('\n====== GENERATION {} ======'.format(idx))
  print(sequence)